In [ ]:
import numpy as np
import pandas as pd

from summer2 import CompartmentalModel
from summer2.parameters import Parameter, DerivedOutput

In [ ]:
relevant_columns = list(range(5))
seir_data = pd.read_csv(
    "./csvs/model_2_1a.csv",
    header=53,
    index_col=0,
    nrows=1001,
    usecols=relevant_columns,
)

In [ ]:
relevant_columns = [0] + list(range(11, 14))
sir_data = pd.read_csv(
    "./csvs/model_2_1a.csv",
    header=53,
    index_col=0,
    nrows=1001,
    usecols=relevant_columns,
)
sir_data.columns = [i_col[:-2] for i_col in sir_data.columns]  # Get rid of the ".1" at the end from duplicate column names

In [ ]:
def build_seir_model(
    config: dict,
) -> CompartmentalModel:
    
    # Model characteristics
    compartments = (
        "Susceptible", 
        "Pre-infectious", 
        "Infectious", 
        "Immune"
    )
    model = CompartmentalModel(
        times=(0., config["end_time"]),
        compartments=compartments,
        infectious_compartments=("Infectious",),
    )
    model.set_initial_population(
        distribution={
            "Susceptible": config["tot_popn"] - config["infectious_seed"],
            "Infectious": config["infectious_seed"],
        }
    )

    # Transitions
    ave_infous = Parameter("ave_infous")
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("R0") / ave_infous,
        source="Susceptible",
        dest="Pre-infectious"
    )
    model.add_transition_flow(
        name="progression", 
        fractional_rate=1. / Parameter("ave_preinfous"),
        source="Pre-infectious", 
        dest="Infectious"
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=1. / ave_infous,
        source="Infectious", 
        dest="Immune"
    )
    return model

In [ ]:
model_config = {
    "tot_popn": 1e5,
    "infectious_seed": 1.,
    "end_time": 1000.,
}

parameters = {
    "ave_preinfous": 2.,
    "ave_infous": 2.,
    "R0": 2.,
}

In [ ]:
seir_model = build_seir_model(model_config)
seir_model.run(parameters=parameters, solver="euler")
derived_outputs = seir_model.get_derived_outputs_df()
outputs = seir_model.get_outputs_df()

In [ ]:
assert abs(seir_data - outputs).max().max() < 1e5

In [ ]:
def build_sir_model(
    config: dict,
) -> CompartmentalModel:
    
    # Model characteristics
    compartments = ("Susceptible", "Infectious", "Immune")
    model = CompartmentalModel(
        times=(0., config["end_time"]),
        compartments=compartments,
        infectious_compartments=["Infectious"],
    )
    model.set_initial_population(
        distribution={
            "Susceptible": config["tot_popn"] - config["infectious_seed"],
            "Infectious": config["infectious_seed"],
        }
    )

    # Transitions
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("R0") / Parameter("ave_infous"),
        source="Susceptible", 
        dest="Infectious"
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=1. / Parameter("ave_infous"), 
        source="Infectious", 
        dest="Immune"
    )
    return model

In [ ]:
sir_model = build_sir_model(model_config)
sir_model.run(parameters=parameters, solver="euler")
sir_compartments = sir_model.get_outputs_df()

In [ ]:
abs(sir_data - sir_compartments).max().max() < 1e5